# For this Capstone project, I will find the safest restaurants to eat at in Boston. I will pull a report of crime in Boston from 2019. This includes Longitude and Latitude information. I will use this data to find which fast food restaurants are in the area using Foresquare. By finding the fast food restaurant that shows up the least, I should have the safest fast food restaurants.

## This is the boston crime data pulled from Kaggle and uploaded. 

In [96]:
# The code was removed by Watson Studio for sharing.

In [97]:
BCrime.shape
dfCrime.shape

(327820, 17)

# Connect to Foursquare to pull fast food information

In [86]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: Hidden 
CLIENT_SECRET: Hidden


# Grab a bunch of restaurants

In [107]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import requests 
search_query = 'Hot Dog Stand'
radius = 10
latitude = float(BCrime['Lat'].head(1).values)
longitude = float(BCrime['Long'].head(1).values)
city = "boston,ma"


#url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url = 'https://api.foursquare.com/v2/venues/search?near={}&client_id={}&client_secret={}&oauth_token={}&v={}&query={}&limit={}'.format(city,CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION,search_query,LIMIT)
results = requests.get(url).json()
#print(results)

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dfRestaurants = pd.json_normalize(venues)
dfRestaurants.head(5)

,id,name,categories,referralId,hasPerk,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.cc,...,location.country,location.formattedAddress,location.address,location.postalCode,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name
0,4c63048beb82d13a60d206d6,Hot Dog Stand,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1621794823,False,Temple place and washington ave,42.358506,-71.058137,"[{'label': 'display', 'lat': 42.35850565365003...",US,...,United States,"[Boston, MA, United States]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4c8135d751ada1cd6c9d1010,Ruggles - Hot Dog Stand,"[{'id': '4bf58dd8d48988d16f941735', 'name': 'H...",v-1621794823,False,NaN,42.336492,-71.089332,"[{'label': 'display', 'lat': 42.33649228533998...",US,...,United States,"[Ruggles Station, Boston, MA 02120, United Sta...",Ruggles Station,02120,NaN,NaN,NaN,NaN,NaN,NaN
2,4f984bd5e4b031f829092d0d,Hot Dog Stand,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1621794823,False,NaN,42.402714,-71.049397,"[{'label': 'display', 'lat': 42.40271436003412...",US,...,United States,"[Somerville, MA 02145, United States]",NaN,02145,NaN,NaN,NaN,NaN,NaN,NaN
3,4f1c697ce4b07beffefa47a7,Corbett's Hot Dog Stand,[],v-1621794823,False,NaN,42.343012,-71.115759,"[{'label': 'display', 'lat': 42.343012, 'lng':...",US,...,United States,"[1209 Beacon St, Brookline, MA 02446, United S...",1209 Beacon St,02446,NaN,NaN,NaN,NaN,NaN,NaN
4,4ebd61d9b8f7b504a77190b5,Hot Dog Stand,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1621794823,False,NaN,42.644784,-71.312803,"[{'label': 'display', 'lat': 42.644784, 'lng':...",US,...,United States,"[Lowell, MA 01852, United States]",NaN,01852,NaN,NaN,NaN,NaN,NaN,NaN


# Find the distance between each restaurant and each crime

In [11]:
pip install haversine

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [108]:
from haversine import haversine, Unit

print("Determining crime rate.")

dfRestaurants['crimecount']=0
for eat,r in dfRestaurants.iterrows():
    for crime,c in dfCrime.iterrows():
        # if a restaurant is less than .1 km from a crime, add to the count
        eatPoint = (dfRestaurants.loc[eat]['location.lat'],dfRestaurants.loc[eat]['location.lng']) # (lat, lon)
        crimePoint = (dfCrime.loc[crime]['Lat'],dfCrime.loc[crime]['Long'])

        if haversine(eatPoint, crimePoint) < .1:
            dfRestaurants.loc[eat,'crimecount'] = dfRestaurants.loc[eat,'crimecount']+1

            
print("Done. This took a while to calculate.")

Determining crime rate.
Done. This took a while to calculate.


# create list of only zero crime restaurants

In [110]:
dfSafe = dfRestaurants[dfRestaurants['crimecount']==0]
print("Safe")
print(dfSafe['name'].head(5))

Safe
1    Ruggles - Hot Dog Stand
2              Hot Dog Stand
3    Corbett's Hot Dog Stand
4              Hot Dog Stand
5         Sals Hot Dog Stand
Name: name, dtype: object


# create a list of only crime restaurants

In [111]:
dfNotSafe = dfRestaurants[dfRestaurants['crimecount']>0]
print("Not Safe")
print(dfNotSafe['name'].head(5))

Not Safe
0                         Hot Dog Stand
10             Liberty Hotel Taxi Stand
11    Park Plaza Hotel Shoe Shine Stand
13                      Rubi's Hot Dogs
14             Western Hotel Taxi Stand
Name: name, dtype: object


In [35]:
pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 5.3 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# My Final Report
## map restaurants (green = safe, red = not safe)

In [112]:
import folium
country_geo = 'world-countries.json'

from branca.element import Figure
fig=Figure(width=650,height=650)

lat = dfSafe['location.lat'].head(1)
lng = dfSafe['location.lng'].head(1)

m1=folium.Map(width=650,height=650,location=[lat,lng],zoom_start=12,min_zoom=4,max_zoom=18)
fig.add_child(m1)
#Adding markers to the map
for safe,r in dfSafe.iterrows():
    lat = dfSafe.loc[safe,'location.lat']
    lng = dfSafe.loc[safe,'location.lng']
    safeName = dfSafe.loc[safe,'name']
    safeAddress = dfSafe.loc[safe,'location.formattedAddress']
    folium.Marker(location=[lat,lng],popup=safeAddress,tooltip=safeName,icon=folium.Icon(color='green',icon='none')).add_to(m1)
for notSafe,r in dfNotSafe.iterrows():
    lat = dfNotSafe.loc[notSafe,'location.lat']
    lng = dfNotSafe.loc[notSafe,'location.lng']
    notSafeName = dfNotSafe.loc[notSafe,'name']
    crimeCount = str(dfNotSafe.loc[notSafe,'crimecount']) + " Crimes in Area"
    folium.Marker(location=[lat,lng],popup=crimeCount,tooltip=notSafeName,icon=folium.Icon(color='red',icon='none')).add_to(m1)

m1



# Above is an interactive map showing hot dogs stands in green that had no crimes nearby and in red if a crime occurred with .1 km.  

# If you click on a red marker, it will show you the number of crimes which occured within .1 km of the hot dog shop. I limited my restaurant style since Foursquare would only give me 30 results.

# In the end, with over 320,000 crimes in the Boston area most of the safe hot dog joints were outside of Boston.

# Ruggles in Boston is the closest and least crime ridden hotdog joint in Boston.

# However, below is the list of safe hot dog joints in and around Boston.


In [117]:
dfSafe.sort_values(by='crimecount', ascending=False).groupby('name')['crimecount'].count()

name
Boston Hot Dog Company                    1
C&R Hot Dogs                              1
Cassie's Hot Dog Cart                     1
Classic Hot Dog Co.                       1
Coffee Pot / Nathan's Famous Hot Dogs     1
Corbett's Hot Dog Stand                   1
Elliots Famous Hot Dogs                   1
Elliott's Hot Dogs                        1
Ernie's Hot Dogs                          1
Greenie's Hot Dog Stand                   1
Harvard Hot Dog                           1
Hometowne Hot Dogs                        1
Hot Diggity Dog                           1
Hot Dog Guy                               1
Hot Dog Scrambler                         1
Hot Dog Stand                             2
Hot Dog Wheels                            1
Hot Dogs & More                           1
Hot Dogs On The Go                        1
Hotdog stand                              1
IKEA                                      1
Indy's Hot Dog Haven                      1
Joe' & Donna's Hot Dog And 